# <center>INSTAGRAM USER ANALYTICS

## IMPORTS AND CONNECTION

Importing packages and files

In [1]:
import pandas as pd
import mysql.connector
from prettytable import PrettyTable
from math import floor, ceil

from sql_credentials import SQL_CONFIG

Eastablishing connection to MySql Database

In [2]:
CONNECTION = mysql.connector.connect(**SQL_CONFIG)
CURSOR = CONNECTION.cursor(buffered=True)

## ANALYSIS

### A) Marketing

#### 1. Rewarding Most Loyal Users
Finding 5 oldest users who have been using Instagram for the longest time.

In [3]:
query = """
    SELECT id, username
    FROM users
    ORDER BY created_at
    LIMIT 5
"""
CURSOR.execute(query)

table = PrettyTable(["ID", "Username"])
for fetched in CURSOR:
    table.add_row([fetched[0], fetched[1]])

print (table)

+----+------------------+
| ID |     Username     |
+----+------------------+
| 80 |   Darby_Herzog   |
| 67 | Emilio_Bernier52 |
| 63 |     Elenor88     |
| 95 |     Nicole71     |
| 38 | Jordyn.Jacobson2 |
+----+------------------+


#### 2. Remind Inactive Users to Start Posting
Finding users who have nerver posted a single photo on Instagram.

In [4]:
query = """
    SELECT id, username
    FROM users
    WHERE id NOT IN (
        SELECT DISTINCT user_id
        FROM photos
    )
"""
CURSOR.execute(query)

table = PrettyTable(["ID", "Username"])
for fetched in CURSOR:
    table.add_row([fetched[0], fetched[1]])

print (table)

+----+---------------------+
| ID |       Username      |
+----+---------------------+
| 5  |    Aniya_Hackett    |
| 7  |  Kasandra_Homenick  |
| 14 |       Jaclyn81      |
| 21 |       Rocio33       |
| 24 |  Maxwell.Halvorson  |
| 25 |    Tierra.Trantow   |
| 34 |        Pearl7       |
| 36 |    Ollie_Ledner37   |
| 41 |      Mckenna17      |
| 45 |   David.Osinski47   |
| 49 |   Morgan.Kassulke   |
| 53 |       Linnea59      |
| 54 |       Duane60       |
| 57 |    Julien_Schmidt   |
| 66 |     Mike.Auer39     |
| 68 |   Franco_Keebler64  |
| 71 |       Nia_Haag      |
| 74 |   Hulda.Macejkovic  |
| 75 |       Leslie67      |
| 76 |  Janelle.Nikolaus81 |
| 80 |     Darby_Herzog    |
| 81 |   Esther.Zulauf61   |
| 83 | Bartholome.Bernhard |
| 89 |     Jessyca_West    |
| 90 |   Esmeralda.Mraz57  |
| 91 |      Bethany20      |
+----+---------------------+


#### 3. Declaring Contest Winner
Finding user with most likes on a single photo

In [5]:
# Sub-query to return image ID with maximum likes
sub_query = """
    SELECT photo_id
    FROM likes
    GROUP BY photo_id
    ORDER BY COUNT(photo_id) DESC
    LIMIT 1
"""
# Fetching user details using the sub-query
query = f"""
    SELECT users.id, users.username, photos.id, photos.image_url
    FROM users INNER JOIN photos
    ON users.id = photos.user_id
    WHERE photos.id = ({sub_query});
"""
CURSOR.execute(query)

table = PrettyTable(["User ID", "Username", "Photo ID", "Image URL"])
for fetched in CURSOR:
    table.add_row([fetched[0], fetched[1], fetched[2], fetched[3]])

print (table)

+---------+---------------+----------+---------------------+
| User ID |    Username   | Photo ID |      Image URL      |
+---------+---------------+----------+---------------------+
|    52   | Zack_Kemmer93 |   145    | https://jarret.name |
+---------+---------------+----------+---------------------+


#### 4. Hashtag Researching
Finding top five commonly used hashtags.

In [6]:
# Sub-query to return Top 5 hashtag counts
sub_query = """
    SELECT COUNT(tag_id)
    FROM photo_tags
    GROUP BY tag_id
    ORDER BY COUNT(tag_id) DESC
    LIMIT 5
"""
# Sub-query to return previous sub-query as MySql8.0 doesn't support IN with LIMIT
sub_query_02 = f"""
    SELECT *
    FROM ({sub_query}) frq_tag_id
"""
# Sub-query to return all the hashtag details which match the Top 5 hashtag counts
sub_query_03 = f"""
    SELECT tag_id, COUNT(tag_id) AS tag_count
	FROM photo_tags
	GROUP BY tag_id
	HAVING tag_count
	IN ({sub_query_02})
"""
# Fetching other details of the hashtags based on previous sub-query
query = f"""
    SELECT tags.id, tags.tag_name, temp_table.tag_count
    FROM tags
    INNER JOIN ({sub_query_03}) AS temp_table
    ON tags.id = temp_table.tag_id
    ORDER BY temp_table.tag_count DESC
"""
CURSOR.execute(query)

table = PrettyTable(["Tag ID", "Tag Name", "Times Used"])
for fetched in CURSOR:
    table.add_row([fetched[0], fetched[1], fetched[2]])

print(table)

+--------+----------+------------+
| Tag ID | Tag Name | Times Used |
+--------+----------+------------+
|   21   |  smile   |     59     |
|   20   |  beach   |     42     |
|   17   |  party   |     39     |
|   13   |   fun    |     38     |
|   18   | concert  |     24     |
|   5    |   food   |     24     |
|   11   |   lol    |     24     |
+--------+----------+------------+


* The above result returns 7 hashtags instead of 5, as the last three tags have same frequency of usage.
* If the partner brand hosts concerts, it will be wise to use #concert as the 5th tag rather than #food or #lol.
* If the partner brand is into food business, choosing #food as the 5th hastag would generate more engagement.
* Lastly, if the partner brand wants to engage into a funny post, it would be relevant to use #lol as the 5th hashtag.

#### 5. Launch AD Campaign
Finding week of the day, to run the AD Campaign, on which most users register.

In [7]:
# Sub-query to return maximum registrations on a day
sub_query = """
    SELECT COUNT(DAYNAME(created_at)) as user_regs
	FROM users
	GROUP BY DAYNAME(created_at)
	ORDER BY user_regs DESC
	LIMIT 1
"""
# Fetching days meeting the maximum registration values
query = f"""
	SELECT DAYNAME(created_at) as dor, COUNT(DAYNAME(created_at)) as user_regs
	FROM users
	GROUP BY dor
	HAVING COUNT(dor) = ({sub_query})
"""
CURSOR.execute(query)

table = PrettyTable(["Day", "Registrations"])
for fetched in CURSOR:
    table.add_row([fetched[0], fetched[1]])

print(table)

+----------+---------------+
|   Day    | Registrations |
+----------+---------------+
| Thursday |       16      |
|  Sunday  |       16      |
+----------+---------------+


* Thursday and Sunday witness the maximum registration, so either of the day can be chosen.

### B) Investor Metrics

#### 1. User Engagement
* Finding how many times does average user post on Instagram.
* Finding total number of photos on Instagram.
* Finding total number of users on Instagram.

In [8]:
# Fetching account creation date and number of images uploaded for each user
query = """
    SELECT DATE(users.created_at) AS doc, users.id AS user_id, COUNT(photos.user_id) AS pics
    FROM users
    LEFT JOIN photos ON users.id = photos.user_id 
    GROUP BY users.id
    ORDER BY doc
"""

sql_df = pd.read_sql_query(sql=query, con=CONNECTION)
sql_df.head()

C:\Users\utkar\AppData\Local\Temp\ipykernel_10676\1169785305.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql_df = pd.read_sql_query(sql=query, con=CONNECTION)


,doc,user_id,pics
0,2016-05-06,67,3
1,2016-05-06,80,0
2,2016-05-08,63,4
3,2016-05-09,95,2
4,2016-05-14,71,0


In [9]:
total_users = len(sql_df)
total_pics = sql_df['pics'].sum()
print(f'Average user posts {floor(total_pics/total_users)} to {ceil(total_pics/total_users)} times on Instagram.')
print(f"There are {total_users} users on Instagram out of which {len(sql_df[sql_df['pics'] == 0.0])} are inactive and have not posted anything.")
print(f"A total of {int(total_pics)} photos have been posted on Instagram till date.")

Average user posts 2 to 3 times on Instagram.
There are 100 users on Instagram out of which 26 are inactive and have not posted anything.
A total of 257 photos have been posted on Instagram till date.


#### 2. Bots & Fake Accounts
* Provide data on Bots
* Bots are those account who have liked every photo on Instagram.

In [10]:
# Sub-query to return total number of images
sub_query = """
    SELECT COUNT(id)
    FROM photos
"""
# Sub-query to return number of images likes by each user
sub_query_02 = """
    SELECT COUNT(photo_id)
    FROM likes
    WHERE likes.user_id = users.id
"""
# Fetching user data where values of sub-query 1 and 2 are equal
query = f"""
	SELECT users.id, users.username
	FROM users
	WHERE ({sub_query}) = ({sub_query_02})
"""
CURSOR.execute(query)

table = PrettyTable(["User ID", "Username"])
for fetched in CURSOR:
    table.add_row([fetched[0], fetched[1]])

print(table)

+---------+--------------------+
| User ID |      Username      |
+---------+--------------------+
|    5    |   Aniya_Hackett    |
|    14   |      Jaclyn81      |
|    21   |      Rocio33       |
|    24   | Maxwell.Halvorson  |
|    36   |   Ollie_Ledner37   |
|    41   |     Mckenna17      |
|    54   |      Duane60       |
|    57   |   Julien_Schmidt   |
|    66   |    Mike.Auer39     |
|    71   |      Nia_Haag      |
|    75   |      Leslie67      |
|    76   | Janelle.Nikolaus81 |
|    91   |     Bethany20      |
+---------+--------------------+


## CONNECTION CLOSE

Clossing connection to MySql Database

In [11]:
CURSOR.close()
CONNECTION.close()